# model_dev.ipynb

Quick test notebook — loads 5 years of historical prices for ASML (ASML.AS).

In [1]:
# --- Run this as the FIRST cell in the notebook ---
import os, sys, pathlib

# 1) Compute project root (parent of the current notebooks folder)
cwd = os.getcwd()
project_root = os.path.abspath(os.path.join(cwd, ".."))  # parent of notebooks

# 2) Change current working directory to project root
os.chdir(project_root)

# 3) Ensure project root and src are first in sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# 4) Print verification
print("changed cwd ->", os.getcwd())
print("sys.path[0:6] ->", sys.path[:6])
print("src_path exists? ->", os.path.exists(src_path))
print("src files ->", list(pathlib.Path(src_path).glob("*.py")))

changed cwd -> /Users/puneetsharma/Desktop/FinanceProject/financial-forecasting-engine
sys.path[0:6] -> ['/Users/puneetsharma/Desktop/FinanceProject/financial-forecasting-engine/src', '/Users/puneetsharma/Desktop/FinanceProject/financial-forecasting-engine', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python39.zip', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/lib-dynload', '']
src_path exists? -> True
src files -> [PosixPath('/Users/puneetsharma/Desktop/FinanceProject/financial-forecasting-engine/src/simulation.py'), PosixPath('/Users/puneetsharma/Desktop/FinanceProject/financial-forecasting-engine/src/__init__.py'), PosixPath('/Users/puneetsharma/Desktop/FinanceProject/financial-forecasting-engine/src/valuation.py'), PosixPath('/Users/puneetsharma/Desktop/FinanceProject/financial-f

In [2]:
import os
os.chdir("/Users/puneetsharma/Desktop/FinanceProject/financial-forecasting-engine")
print("cwd fixed to:", os.getcwd())

cwd fixed to: /Users/puneetsharma/Desktop/FinanceProject/financial-forecasting-engine


In [3]:
# Week 1 — Deterministic Forecasting for ASML (5-year)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from datetime import datetime


os.makedirs("outputs/forecasts", exist_ok=True)

plt.rcParams["figure.figsize"] = (9,5)

In [4]:
import yfinance as yf
import pandas as pd
from src.load_data import load_price_series

ticker = "ASML.AS"
df = load_price_series(ticker, period='6y')
df.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2025-11-17 00:00:00+01:00,879.299988,883.900024,864.299988,876.400024,471563,0.0,0.0
2025-11-18 00:00:00+01:00,872.000000,877.500000,857.099976,865.599976,593874,0.0,0.0
2025-11-19 00:00:00+01:00,861.099976,895.200012,857.299988,886.599976,569198,0.0,0.0
2025-11-20 00:00:00+01:00,908.799988,914.700012,889.799988,889.799988,673609,0.0,0.0
2025-11-21 00:00:00+01:00,839.200012,859.299988,822.200012,834.000000,976903,0.0,0.0


In [5]:
# ============================
# WEEK 1 — STEP 2
# Deterministic ASML Forecast Using forecasting.py
# ============================

from src.forecasting import compute_cagr, build_forecast
import pandas as pd

# --------------------------------------------
# 1) ASML Historical Revenue (2019–2024)
# --------------------------------------------
hist_revenue = {
    2019: 11820,
    2020: 13978.5,
    2021: 18611,
    2022: 21173.4,
    2023: 27558.5 ,
    2024: 28262.9
}

revenue_values = list(hist_revenue.values())
periods = len(revenue_values) - 1   # 2019 → 2024 = 6 periods

# --------------------------------------------
# 2) Compute historical CAGR
# --------------------------------------------
cagr = compute_cagr(revenue_values, periods)
print("ASML Revenue CAGR 2019–2024:", round(cagr*100, 2), "%")


ebitda_margin = 0.34935
capex_pct = 0.07190
dep_pct = 0.03124
wc_pct = 0.02924
tax_rate = 0.16472
start_year = 2024

# --------------------------------------------
# 3) Build Base Case Forecast (6-year)
# --------------------------------------------
last_revenue = revenue_values[-1]

forecast_base = build_forecast(
    last_revenue=last_revenue,
    growth=cagr,
    ebitda_margin=0.34935,
    capex_pct=0.07190,
    dep_pct=0.03124,
    wc_pct=0.02924,
    tax_rate=0.16472,
    years=5,
    start_year=2024   
)

print("\nBASE CASE FORECAST:")
display(forecast_base.round(0))

# --------------------------------------------
# 5) BULL CASE (25% higher growth, +2% margin)
# --------------------------------------------
forecast_bull = build_forecast(
    last_revenue=last_revenue,
    growth=cagr * 1.25,
    ebitda_margin= ebitda_margin + 0.02,
    capex_pct=capex_pct * 1.05,
    dep_pct=dep_pct,
    wc_pct=wc_pct,
    tax_rate=tax_rate,
    years=5,
    start_year=start_year
)

print("\n=== BULL CASE FORECAST ===")
display(forecast_bull.round(1))

# --------------------------------------------
# 6) BEAR CASE (25% lower growth, −3% margin)
# --------------------------------------------
forecast_bear = build_forecast(
    last_revenue=last_revenue,
    growth=cagr * 0.75,
    ebitda_margin=ebitda_margin - 0.03,
    capex_pct=capex_pct * 0.95,
    dep_pct=dep_pct,
    wc_pct=wc_pct * 1.1,
    tax_rate=tax_rate,
    years=5,
    start_year=start_year
)

print("\n=== BEAR CASE FORECAST ===")
display(forecast_bear.round(1))

# --------------------------------------------
# 7) Export CSVs — required for Week 2 (DCF + Monte Carlo) & Week 4 (Power BI)
# --------------------------------------------
forecast_base.to_csv("outputs/forecasts/base_case.csv", index_label="Year")
forecast_bull.to_csv("outputs/forecasts/bull_case.csv", index_label="Year")
forecast_bear.to_csv("outputs/forecasts/bear_case.csv", index_label="Year")

print("\nSaved forecast CSVs to outputs/forecasts/")

ASML Revenue CAGR 2019–2024: 19.05 %

BASE CASE FORECAST:


,Revenue,EBITDA,Depreciation,EBIT,NOPAT,Capex,ΔWorkingCapital,FCFF
2025,33646.0,11754.0,1051.0,10703.0,8940.0,2419.0,984.0,6588.0
2026,40055.0,13993.0,1251.0,12742.0,10643.0,2880.0,1171.0,7843.0
2027,47684.0,16659.0,1490.0,15169.0,12670.0,3429.0,1394.0,9337.0
2028,56767.0,19832.0,1773.0,18058.0,15084.0,4082.0,1660.0,11116.0
2029,67580.0,23609.0,2111.0,21498.0,17957.0,4859.0,1976.0,13233.0



=== BULL CASE FORECAST ===


,Revenue,EBITDA,Depreciation,EBIT,NOPAT,Capex,ΔWorkingCapital,FCFF
2025,34992.1,12924.3,1093.2,11831.2,9882.3,2641.7,1023.2,7310.6
2026,43323.4,16001.5,1353.4,14648.1,12235.3,3270.7,1266.8,9051.2
2027,53638.4,19811.3,1675.7,18135.7,15148.4,4049.4,1568.4,11206.2
2028,66409.3,24528.3,2074.6,22453.6,18755.1,5013.6,1941.8,13874.3
2029,82220.8,30368.3,2568.6,27799.7,23220.5,6207.3,2404.1,17177.7



=== BEAR CASE FORECAST ===


,Revenue,EBITDA,Depreciation,EBIT,NOPAT,Capex,ΔWorkingCapital,FCFF
2025,32300.4,10315.1,1009.1,9306.1,7773.2,2206.3,1038.9,5537.0
2026,36914.7,11788.7,1153.2,10635.5,8883.6,2521.5,1187.3,6328.0
2027,42188.2,13472.8,1318.0,12154.8,10152.7,2881.7,1356.9,7232.0
2028,48215.0,15397.4,1506.2,13891.2,11603.1,3293.3,1550.8,8265.2
2029,55102.7,17597.1,1721.4,15875.6,13260.6,3763.8,1772.3,9445.9



Saved forecast CSVs to outputs/forecasts/


In [6]:
# -------------------------
# Week 2 — Run DCF + Monte Carlo
# -------------------------
import numpy as np
import pandas as pd
from src.forecasting import build_forecast
from src.valuation import compute_dcf_value, wacc_calc
from src.simulation import run_mc_simulation
from src.load_data import load_price_series

# Config
n_sims = 10000   # reduce for testing; set 10000 for final run (may be slow)
years = 5
start_year = 2024   # must match Week 1 start_year
forecasts_dir = "outputs/forecasts"
out_dir = "outputs/mc"
PathExists = __import__("os").path.exists
import os
os.makedirs(out_dir, exist_ok=True)

# Load base forecast CSV (we'll reconstruct base inputs from it)
base_df = pd.read_csv(f"{forecasts_dir}/base_case.csv")
# infer last_revenue from first row or last row? Use the last historical revenue we used earlier
# Here we read the first forecast year's Revenue as the first projected year; but we want last actual revenue:
# we'll reconstruct last_revenue as base_df['Revenue'].iloc[0]/(1+growth) if you know growth, simpler: ask user-supplied last_revenue
# For safety, pull last_revenue from your hist_revenue if available; else set from base_df.
try:
    last_revenue = float(base_df["Revenue"].iloc[0]) / (1 + 0)  # if base inputs not stored, fallback
except Exception:
    last_revenue = float(base_df["Revenue"].iloc[0])

# Base inputs (use the same real averages you defined in Week1)
base_inputs = {
    "last_revenue": last_revenue,
    "growth": float(cagr),
    "ebitda_margin": 0.34935,
    "capex_pct": 0.07190,
    "dep_pct": 0.03124,
    "wc_pct": 0.02924,
    "tax_rate": 0.16472
}

# sigma (std dev) 
sigma_inputs = {
    "growth": 0.05 * abs(base_inputs["growth"]) if base_inputs["growth"]!=0 else 0.05,
    "ebitda_margin": 0.03,   # +/- 3ppt
    "capex_pct": 0.02,
    "dep_pct": 0.01,
    "wc_pct": 0.01,
    "tax_rate": 0.02
}

# WACC & terminal g assumptions
wacc_base = 0.0725  # Average of wacc formula and market estimates.
wacc_sigma = 0.015
terminal_g_base = 0.03
terminal_g_sigma = 0.005

# Run Monte Carlo 
vals = run_mc_simulation(
    n_sims=n_sims,
    base_inputs=base_inputs,
    sigma_inputs=sigma_inputs,
    years=years,
    start_year=start_year,
    wacc_base=wacc_base,
    wacc_sigma=wacc_sigma,
    terminal_g_base=terminal_g_base,
    terminal_g_sigma=terminal_g_sigma,
    verbose=False
)

# Save raw values
np.save(f"{out_dir}/mc_values.npy", vals)

# Summarize
clean_vals = vals[~np.isnan(vals)]
quantiles = np.percentile(clean_vals, [5,25,50,75,95])
summary = {
    "n_sims": int(n_sims),
    "n_success": int(len(clean_vals)),
    "5%": float(quantiles[0]),
    "25%": float(quantiles[1]),
    "50%_median": float(quantiles[2]),
    "75%": float(quantiles[3]),
    "95%": float(quantiles[4]),
    "mean": float(np.mean(clean_vals)),
    "std": float(np.std(clean_vals))
}
summary_df = pd.DataFrame([summary])
summary_df.to_csv(f"{out_dir}/mc_summary.csv", index=False)

# Prob intrinsic > market price
# try to get latest market price per share using load_price_series (if available)
try:
    price_df = load_price_series("ASML.AS", period="1d")
    market_price = float(price_df["Close"].iloc[-1])
except Exception:
    market_price = None

# If market_price available, compute probability intrinsic > market
# Need to convert enterprise value -> per-share intrinsic: requires shares outstanding or market cap.
# We'll compute probability_intrinsic_greater only if user supplies market_cap and shares (optional).
prob_above = None

print("Monte Carlo finished.")
print("Summary:")
display(summary_df)
print("Raw values saved to:", f"{out_dir}/mc_values.npy")
print("Summary saved to:", f"{out_dir}/mc_summary.csv")
if market_price is not None:
    print("Market price (ASML.AS) latest close:", market_price)
else:
    print("Market price not fetched; supply market cap/shares for probability calc.")

Monte Carlo finished.
Summary:


,n_sims,n_success,5%,25%,50%_median,75%,95%,mean,std
0,10000,9971,169856.292822,239444.135628,311965.149647,430240.890142,843561.273016,416638.723392,880022.460724


Raw values saved to: outputs/mc/mc_values.npy
Summary saved to: outputs/mc/mc_summary.csv
Market price (ASML.AS) latest close: 834.0


In [7]:


# --- WEEK 2 — Final Metrics (Clean EUR Version) ---

import numpy as np
import pandas as pd
import json, os

# Load canonical EUR file
ev_path = "outputs/mc/mc_values_live_eur.npy"
ev_eur = np.load(ev_path)
clean = ev_eur[~np.isnan(ev_eur)]

# Percentiles
qs = np.percentile(clean, [5,25,50,75,95])

metrics = {
    "n_sims": int(len(ev_eur)),
    "n_success": int(len(clean)),
    "mean_ev_eur": float(clean.mean()),
    "std_ev_eur": float(clean.std()),
    "ev_5pct_eur": float(qs[0]),
    "ev_25pct_eur": float(qs[1]),
    "ev_median_eur": float(qs[2]),
    "ev_75pct_eur": float(qs[3]),
    "ev_95pct_eur": float(qs[4]),
}

# Per-share (shares in units)
shares_outstanding_units = 388150000.0

metrics["median_per_share_eur"] = float(qs[2] / shares_outstanding_units)
metrics["mean_per_share_eur"]   = float(clean.mean() / shares_outstanding_units)

# Probability intrinsic > market price
market_price = 1000   # OPTIONAL: update to latest ASML price
metrics["prob_intrinsic_greater_than_market"] = float(
    (clean / shares_outstanding_units > market_price).mean()
)

# CVaR 5%
k = max(1,int(0.05*len(clean)))
metrics["cvar_5pct_ev_eur"] = float(np.sort(clean)[:k].mean())

# Save
with open("outputs/mc/mc_metrics.json","w") as f:
    json.dump(metrics, f, indent=2)

pd.DataFrame([metrics]).to_csv("outputs/mc/mc_metrics.csv", index=False)

print("=== Clean EUR Metrics Saved ===")
display(pd.DataFrame([metrics]))

=== Clean EUR Metrics Saved ===


,n_sims,n_success,mean_ev_eur,std_ev_eur,ev_5pct_eur,ev_25pct_eur,ev_median_eur,ev_75pct_eur,ev_95pct_eur,median_per_share_eur,mean_per_share_eur,prob_intrinsic_greater_than_market,cvar_5pct_ev_eur
0,10000,9853,5.965637e+11,1.725123e+12,1.928253e+11,2.786851e+11,3.813255e+11,5.544018e+11,1.316574e+12,982.417993,1536.941011,0.485943,1.672448e+11


In [8]:
import numpy as np, os

base = "outputs/mc"
# files to check (adjust names if your files differ)
files = {
    "original_mc": os.path.join(base, "mc_values.npy"),
    "live_mc_raw": os.path.join(base, "mc_values_live.npy"),
    "live_mc_eur": os.path.join(base, "mc_values_live_eur.npy")
}

for name, path in files.items():
    if os.path.exists(path):
        arr = np.load(path)
        arr_clean = arr[~np.isnan(arr)]
        print(name, "exists:", path, 
              "count:", len(arr_clean),
              "median:", np.median(arr_clean),
              "mean:", np.mean(arr_clean),
              "min:", arr_clean.min(), "max:", arr_clean.max())
    else:
        print(name, "missing:", path)

original_mc exists: outputs/mc/mc_values.npy count: 9971 median: 311965.1496470694 mean: 416638.723392108 min: 67611.84509955462 max: 59262290.6794199
live_mc_raw exists: outputs/mc/mc_values_live.npy count: 9853 median: 381325.54402828176 mean: 596563.6535985407 min: 95915.54565306078 max: 94188516.69478595
live_mc_eur exists: outputs/mc/mc_values_live_eur.npy count: 9853 median: 381325544028.28174 mean: 596563653598.5408 min: 95915545653.06078 max: 94188516694785.95


In [9]:
shares_units = 388150000.0    # units
shares_millions = 388.15      # millions

# pick a file you saw exists from step 1, e.g. "live_mc_eur" or "original_mc"
path = os.path.join("outputs","mc","mc_values_live_eur.npy")  # change to the file that exists
arr = np.load(path)
arr = arr[~np.isnan(arr)]

print("median EV (as-stored):", np.median(arr))

# if arr is EV in millions -> convert to euros
print("median per-share if arr is millions (EUR/share):", np.median(arr)/shares_millions)
# if arr is EV in EUR -> convert using shares in units
print("median per-share if arr is EUR (EUR/share):", np.median(arr)/shares_units)

median EV (as-stored): 381325544028.28174
median per-share if arr is millions (EUR/share): 982417993.1167893
median per-share if arr is EUR (EUR/share): 982.4179931167893


In [10]:
# === Overwrite canonical MC with last-run `vals` (safe backup first) ===
import os, shutil, json
import numpy as np, pandas as pd
from datetime import datetime

# make sure `vals` exists
try:
    _ = vals
except NameError:
    raise RuntimeError("`vals` not found in notebook memory. Run the MC cell first to produce `vals`.")

out_dir = "outputs/mc"
os.makedirs(out_dir, exist_ok=True)
bk_dir = os.path.join(out_dir, "backups")
os.makedirs(bk_dir, exist_ok=True)

ts = datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")

# backup any existing canonical files
for fname in ["mc_values_live.npy", "mc_values_live_eur.npy", "mc_last_run.csv", "mc_metrics.json"]:
    p = os.path.join(out_dir, fname)
    if os.path.exists(p):
        shutil.copy2(p, os.path.join(bk_dir, f"{fname}.{ts}.bak"))

# Save canonical: vals is EV in MILLIONS
np.save(os.path.join(out_dir, "mc_values_live.npy"), vals)
ev_eur = vals * 1_000_000.0
np.save(os.path.join(out_dir, "mc_values_live_eur.npy"), ev_eur)

# Compute metrics (on full-EUR clean data)
clean = ev_eur[~np.isnan(ev_eur)]
qs = np.percentile(clean, [5,25,50,75,95])
shares_units = 388150000.0   # make sure this matches the units you use in Streamlit
market_price = 869.20        # update if you want

metrics = {
    "run_at": datetime.utcnow().isoformat(),
    "n_sims": int(len(vals)),
    "n_valid": int(clean.size),
    "ev_mean_eur": float(np.nanmean(clean)),
    "ev_std_eur": float(np.nanstd(clean)),
    "ev_5pct_eur": float(qs[0]),
    "ev_25pct_eur": float(qs[1]),
    "ev_median_eur": float(qs[2]),
    "ev_75pct_eur": float(qs[3]),
    "ev_95pct_eur": float(qs[4]),
    "median_per_share_eur": float(qs[2] / shares_units),
    "mean_per_share_eur": float(np.nanmean(clean) / shares_units),
    "prob_intrinsic_greater_than_market": float(((clean / shares_units) > market_price).mean()),
    "cvar_5pct_ev_eur": float(np.sort(clean)[: max(1,int(0.05*clean.size))].mean())
}

# write metrics
with open(os.path.join(out_dir, "mc_metrics.json"), "w") as f:
    json.dump(metrics, f, indent=2)
pd.DataFrame([metrics]).to_csv(os.path.join(out_dir, "mc_last_run.csv"), index=False)

print("Backups (if any) written to:", bk_dir)
print("Canonical files overwritten with current `vals` run.")
print("New metrics summary:")
for k,v in metrics.items():
    print(f"  {k}: {v}")

Backups (if any) written to: outputs/mc/backups
Canonical files overwritten with current `vals` run.
New metrics summary:
  run_at: 2025-11-21T17:19:34.444264
  n_sims: 10000
  n_valid: 9971
  ev_mean_eur: 416638723392.108
  ev_std_eur: 880022460724.452
  ev_5pct_eur: 169856292822.2799
  ev_25pct_eur: 239444135627.82404
  ev_median_eur: 311965149647.0694
  ev_75pct_eur: 430240890141.9802
  ev_95pct_eur: 843561273016.4886
  median_per_share_eur: 803.7231731213948
  mean_per_share_eur: 1073.3961700170244
  prob_intrinsic_greater_than_market: 0.42924480994885167
  cvar_5pct_ev_eur: 146066824897.8496
